In [36]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT


from tools.classical import *
from tools.quantum import *

In [37]:
num_qubits = 4
num_estimating_qubits = 4

X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()

# print(hamiltonian_qt)
#* HAMILTONIANS
# rescaling_factor, shift = rescaling_and_shift_factors(hamiltonian_qt)  # Shift already rescaled
# just_shift = shift * rescaling_factor                                  # Unscaled shift
# shifted_hamiltonian_qt = hamiltonian_qt + just_shift * qt.qeye(hamiltonian_qt.shape[0])
# rescaled_hamiltonian_qt = hamiltonian_qt / rescaling_factor + shift * qt.qeye(hamiltonian_qt.shape[0])

### Find Heisenberg coefficients for non degenerate spectrum

In [38]:
coeff_lower_bound = 0
coeff_xx = np.arange(1, coeff_lower_bound, -0.1)
coeff_yy = np.arange(1, coeff_lower_bound, -0.1)
coeff_zz = np.arange(1, coeff_lower_bound, -0.1)

# get all combinations of coefficients
coeffs = np.array(np.meshgrid(coeff_xx, coeff_yy, coeff_zz)).T.reshape(-1, 3)

for coeff in coeffs:
    hamiltonian_qt = hamiltonian_matrix([X, X], [Y, Y], [Z, Z], coeffs=coeff, num_qubits=num_qubits, sym_break_term=[Z])
    exact_spec = np.linalg.eigvalsh(hamiltonian_qt)
    exact_spec = np.round(exact_spec, 3)

    if len(np.unique(exact_spec)) == len(exact_spec):
        print("Unique spectrum found: ", exact_spec)
        print("Coefficients: ", coeff)
        break


Unique spectrum found:  [-8.086 -5.236 -4.    -3.236 -1.486 -1.236 -0.764 -0.     0.764  1.225
  1.236  2.     3.236  4.348  5.236  6.   ]
Coefficients:  [1. 1. 1.]


In [39]:

#* Energy register check for QPE, non-degenerate and non-rescaled but chosen E = 0.764
hamiltonian_qt = hamiltonian_matrix([X, X], [Y, Y], [Z, Z], coeffs=[1, 1, 1], num_qubits=num_qubits, sym_break_term=[Z])
exact_spec, eigenstates = np.linalg.eigh(hamiltonian_qt)

eigenstate = eigenstates[8]
initial_energy = exact_spec[8]
initial_eigenstate = Statevector(eigenstate.reshape(2**num_qubits, 1))
print(f'Initial energy = {initial_energy}')

# Initialize
qr_energy = QuantumRegister(num_estimating_qubits, name="E")
cr_energy = ClassicalRegister(num_estimating_qubits, name="crE")
qr_sys  = QuantumRegister(num_qubits, name="sys")
circ = QuantumCircuit(qr_energy, qr_sys, cr_energy)
circ.initialize(initial_eigenstate, qr_sys)
circ.h(qr_energy)

# QPE
T = 1
total_time = T * 2 * np.pi
num_trotter_steps = 100
step_size = total_time / num_trotter_steps

# Double the num_trotter_steps for T = 2 and so on...
trotter_step_circ = trotter_step_heisenberg(num_qubits)
ham_time_evol = lambda m, t: ham_evol(num_qubits, trotter_step=trotter_step_circ, num_trotter_steps=m, time=t)
U = ham_time_evol(num_trotter_steps, total_time)
cU = U.control(1)
for q in range(num_estimating_qubits):
    for _ in range(2**q):
        circ.compose(cU, [q, *list(qr_sys)], inplace=True)

# Inverse QFT
qft_circ = QFT(num_estimating_qubits, inverse=True)  # With SWAPs
circ.compose(qft_circ, qr_energy, inplace=True)

circ.measure(qr_energy, cr_energy)
print(circ)

Initial energy = 0.7639320225002102


In [ ]:
tr_circ = transpile(circ, basis_gates=['u', 'cx'], optimization_level=3)
simulator = Aer.get_backend('statevector_simulator')
shots = 10
job = simulator.run(tr_circ, shots=shots)
counts = job.result().get_counts()
print(counts)

#! For non rescaled spectrum, under 100 Trotter steps we have garbage as we found out!


{'0111': 4, '0101': 8, '0110': 8, '1011': 8, '1110': 4, '1010': 3, '1111': 9, '1101': 4, '0011': 7, '0001': 6, '1100': 5, '0100': 7, '0000': 4, '0010': 7, '1000': 6, '1001': 10}
